In [6]:
import torch
from torch import nn
import sys
import json
import os
import multiprocessing


sys.path.append(".")
sys.path.append("..")
sys.path.append("../..")

from d2lzh_pytorch.BERT_unit_v3.use_BERT import load_pretrained_model, \
SNLIBERTDataset, BERTClassifier, load_data


from d2lzh_pytorch.myUtils import try_all_gpus

In [12]:
# 导入模型
from d2lzh_pytorch.BERT_unit.BERT_model import BERTModel


vocab_len = 60005

num_hiddens = 256

ffn_num_hiddens = 512

num_heads = 4
num_layers = 2
dropout = 0.1
max_len = 512

In [68]:

# bert模型
bert = BERTModel(vocab_size=60005,
                 num_hiddens=768,
                 norm_shape=[768],
                 
                 ffn_num_input=768,
                 ffn_num_hiddens=3072,
                 
                 num_heads=4,
                 num_layers=2,
                 dropout=0.2,
                 max_len=512,
                 
                 key_size=768,
                 query_size=768,
                 value_size=768,
                 
                 hid_in_features=768,
                 mlm_in_features=768,
                 nsp_in_features=768
                )

## 加载 bert.base 模型参数

In [69]:
# 路径配置
base_path = r"/home/mylady/code/python/DL-pytorch/apps/chapter_pytorch_demo/data"
target_path = r"bert.base.torch/pretrained.params"

data_dir = os.path.join(base_path, target_path)

In [52]:
a = torch.load(data_dir)


In [ ]:


# 加载
bert.load_state_dict(torch.load(data_dir))

In [24]:

# 加载数据
train_iter, test_iter = load_data(vocab)

read 549367 examples
read 9824 examples


In [25]:
len(train_iter), len(test_iter)

(1073, 20)

打印 train_iter 数据看看

In [31]:

# 打印一下数据
count = 0
for X, Y in train_iter:
    
    print(X[0].shape)  # [512, 128]
    print(X[1].shape)  # [512, 128]
    print(Y.shape)     # [512]
    
    count += 1
    
    if count >= 3:
        print('end: ', count)
        break
    print('count: ', count)
    print("")

torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512])
count:  1

torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512])
count:  2

torch.Size([512, 128])
torch.Size([512, 128])
torch.Size([512])
end:  3


In [33]:
# 模型及参数
net = BERTClassifier(bert)
lr = 1e-4
num_epochs = 5


trainer = torch.optim.Adam(net.parameters(), lr=lr)
loss = nn.CrossEntropyLoss(reduction='none')

In [ ]:
# 如果出现显存不足错误，请减少“batch_size”。在原始的BERT模型中，max_len=512
batch_size = 512
max_len = 128
num_workers = 4


# 开始微调
train_ch13(net,
           train_iter, 
           test_iter,
           loss, 
           trainer,
           num_epochs,
           devices)